In [91]:
import pandas as pd

clients = pd.read_csv("IB_Clients.csv")
clients.head()

,Client_ID,Client_Name,Client_Segment,Country
0,1102,Jared Diaz DDS,Retail,USA
1,1435,Michael Jones,Institutional,USA
2,1860,Christina Huang,Retail,NaN
3,1270,Philip Key,Hign,NaN
4,1106,Allison Henderson MD,Institutional,India


In [2]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Client_ID       1020 non-null   int64 
 1   Client_Name     970 non-null    object
 2   Client_Segment  1020 non-null   object
 3   Country         853 non-null    object
dtypes: int64(1), object(3)
memory usage: 32.0+ KB


In [3]:
clients["Client_ID"].duplicated().sum()

np.int64(20)

In [15]:
clients_clean = clients.drop_duplicates(
    subset="Client_ID",
    keep="first"
).copy()

In [20]:
clients_clean["Client_Segment"] = clients_clean["Client_Segment"].replace({
    "Retaill": "Retail",
    "Hign": "HNI"
})

In [21]:
clients_clean["Client_Segment"].value_counts()

Client_Segment
HNI              419
Retail           399
Institutional    182
Name: count, dtype: int64

In [22]:
clients_clean["Client_Name"] = clients_clean["Client_Name"].fillna("Unknown_Client")

In [23]:
clients_clean["Client_Name"].isna().sum()

np.int64(0)

In [24]:
clients_clean["Country"] = clients_clean["Country"].replace("UAS", "USA")
clients_clean["Country"] = clients_clean["Country"].fillna("Unknown")

In [25]:
clients_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Client_ID       1000 non-null   int64 
 1   Client_Name     1000 non-null   object
 2   Client_Segment  1000 non-null   object
 3   Country         1000 non-null   object
dtypes: int64(1), object(3)
memory usage: 39.1+ KB


In [26]:
portfolios = pd.read_csv("IB_Portfolios.csv")

In [27]:
portfolios.head()

,Portfolio_ID,Client_ID,Risk_Level,Portfolio_Value_USD
0,5522,1102.0,Medium,534591.99
1,5619,1435.0,Medium,157662.40
2,5924,1860.0,Low,81430.50
3,5262,1270.0,High,205610.76
4,5873,1106.0,Low,137493.51


In [29]:
portfolios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010 entries, 0 to 1009
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Portfolio_ID         1010 non-null   int64  
 1   Client_ID            1000 non-null   float64
 2   Risk_Level           1010 non-null   object 
 3   Portfolio_Value_USD  1010 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 31.7+ KB


In [30]:
portfolios["Portfolio_ID"].duplicated().sum()

np.int64(10)

In [31]:
portfolios_clean = portfolios.drop_duplicates(
    subset="Portfolio_ID",
    keep="first"
).copy()

In [32]:
portfolios_clean["Portfolio_ID"].duplicated().sum()

np.int64(0)

In [33]:
portfolios_clean["Client_ID"].isna().sum()

np.int64(10)

In [34]:
portfolios_clean["Client_ID"] = portfolios_clean["Client_ID"].fillna(-1)

In [35]:
portfolios_clean["Client_ID"].isna().sum()

np.int64(0)

In [36]:
portfolios_clean["Risk_Level"].value_counts()

Risk_Level
High      257
Hign      257
Medium    253
Low       233
Name: count, dtype: int64

In [37]:
portfolios_clean["Risk_Level"] = portfolios_clean["Risk_Level"].replace({
    "Hign": "High"
})

In [38]:
portfolios_clean["Risk_Level"].value_counts()

Risk_Level
High      514
Medium    253
Low       233
Name: count, dtype: int64

In [39]:
portfolios_clean["Portfolio_Value_USD"].describe()

count    1.000000e+03
mean     2.462886e+05
std      2.409676e+05
min     -4.344144e+04
25%      5.377310e+04
50%      1.737786e+05
75%      3.809751e+05
max      2.029556e+06
Name: Portfolio_Value_USD, dtype: float64

In [40]:
(portfolios_clean["Portfolio_Value_USD"] < 0).sum()

np.int64(34)

In [41]:
portfolios_clean["Portfolio_Value_USD"] = portfolios_clean["Portfolio_Value_USD"].abs()

In [42]:
(portfolios_clean["Portfolio_Value_USD"] < 0).sum()

np.int64(0)

In [44]:
#Merging Clients And Portfolio
client_portfolio = portfolios_clean.merge(
    clients_clean,
    on="Client_ID",
    how="left"
)

In [45]:
client_portfolio.head()

,Portfolio_ID,Client_ID,Risk_Level,Portfolio_Value_USD,Client_Name,Client_Segment,Country
0,5522,1102.0,Medium,534591.99,Jared Diaz DDS,Retail,USA
1,5619,1435.0,Medium,157662.40,Michael Jones,Institutional,USA
2,5924,1860.0,Low,81430.50,Christina Huang,Retail,Unknown
3,5262,1270.0,High,205610.76,Philip Key,HNI,Unknown
4,5873,1106.0,Low,137493.51,Allison Henderson MD,Institutional,India


In [46]:
client_exposure = client_portfolio.groupby("Client_ID")["Portfolio_Value_USD"].sum().reset_index()

In [47]:
client_exposure.head()

,Client_ID,Portfolio_Value_USD
0,-1.0,3.755385e+06
1,1000.0,3.892193e+05
2,1001.0,9.499288e+05
3,1004.0,2.359229e+05
4,1006.0,2.133080e+05


In [48]:
client_exposure.sort_values(
    by="Portfolio_Value_USD",
    ascending=False
).head(5)

,Client_ID,Portfolio_Value_USD
0,-1.0,3.755385e+06
304,1488.0,2.205576e+06
343,1549.0,2.029556e+06
587,1957.0,2.008037e+06
44,1074.0,1.584349e+06


In the above table -1 represent Unmapped Potfolio(Missing Client_ID).Therefore the bank has #3.7 exposure thaqt is not mapped to any client

In [49]:
client_exposure_valid = client_exposure[
    client_exposure["Client_ID"] != -1
]

In [50]:
client_exposure_valid.sort_values(
    by="Portfolio_Value_USD",
    ascending=False
).head(5)

,Client_ID,Portfolio_Value_USD
304,1488.0,2.205576e+06
343,1549.0,2.029556e+06
587,1957.0,2.008037e+06
44,1074.0,1.584349e+06
143,1224.0,1.497700e+06


In [51]:
risk_exposure = portfolios_clean.groupby("Risk_Level")["Portfolio_Value_USD"].agg([
    "count",
    "sum",
    "mean"
]).reset_index()

risk_exposure

,Risk_Level,count,sum,mean
0,High,514,1.264997e+08,246108.339983
1,Low,233,5.632077e+07,241720.030155
2,Medium,253,6.423470e+07,253892.078255


In [52]:
# Merging  clients And portfolios first
client_portfolio = portfolios_clean.merge(
    clients_clean[["Client_ID", "Client_Segment"]],
    on="Client_ID",
    how="left"
)

# Now cross analysis
segment_risk = pd.crosstab(
    client_portfolio["Client_Segment"],
    client_portfolio["Risk_Level"]
)

segment_risk

Risk_Level,High,Low,Medium
Client_Segment,,,
HNI,209,94,88
Institutional,90,34,50
Retail,210,103,112


In [53]:
segment_risk_percent = segment_risk.div(segment_risk.sum(axis=1), axis=0) * 100
segment_risk_percent.round(2)

Risk_Level,High,Low,Medium
Client_Segment,,,
HNI,53.45,24.04,22.51
Institutional,51.72,19.54,28.74
Retail,49.41,24.24,26.35


In [54]:
# Merging portfolios with clients to get segment + value together
merged = portfolios_clean.merge(
    clients_clean[["Client_ID", "Client_Segment"]],
    on="Client_ID",
    how="left"
)

# Sorting by portfolio value
merged_sorted = merged.sort_values("Portfolio_Value_USD", ascending=False)

# Top 5% clients
top_5_percent = int(len(merged_sorted) * 0.05)

top_exposure = merged_sorted.head(top_5_percent)

total_capital = merged_sorted["Portfolio_Value_USD"].sum()
top_capital = top_exposure["Portfolio_Value_USD"].sum()

(top_capital / total_capital) * 100

np.float64(17.571765942779294)

In [57]:
merged.columns

Index(['Portfolio_ID', 'Client_ID', 'Risk_Level', 'Portfolio_Value_USD',
       'Client_Segment'],
      dtype='object')

In [58]:
def stress_loss(row):
    if row["Risk_Level"] == "High":
        return row["Portfolio_Value_USD"] * 0.25
    elif row["Risk_Level"] == "Medium":
        return row["Portfolio_Value_USD"] * 0.15
    else:
        return row["Portfolio_Value_USD"] * 0.05

merged["Stress_Loss"] = merged.apply(stress_loss, axis=1)

In [59]:
"Stress_Loss" in merged.columns

True

In [60]:
segment_stress = merged.groupby("Client_Segment").agg({
    "Portfolio_Value_USD": "sum",
    "Stress_Loss": "sum"
})

segment_stress["Stress_Loss_%"] = (
    segment_stress["Stress_Loss"] /
    segment_stress["Portfolio_Value_USD"]
) * 100

segment_stress

,Portfolio_Value_USD,Stress_Loss,Stress_Loss_%
Client_Segment,,,
HNI,9.200941e+07,1.698341e+07,18.458334
Institutional,4.144352e+07,7.787789e+06,18.791330
Retail,1.098468e+08,1.860414e+07,16.936436


In [62]:
merged.head()

,Portfolio_ID,Client_ID,Risk_Level,Portfolio_Value_USD,Client_Segment,Stress_Loss
0,5522,1102.0,Medium,534591.99,Retail,80188.7985
1,5619,1435.0,Medium,157662.40,Institutional,23649.3600
2,5924,1860.0,Low,81430.50,Retail,4071.5250
3,5262,1270.0,High,205610.76,HNI,51402.6900
4,5873,1106.0,Low,137493.51,Institutional,6874.6755


In [63]:
import numpy as np

np.random.seed(42)

def simulate_returns(row):
    if row["Risk_Level"] == "High":
        return np.random.normal(0.001, 0.02)
    elif row["Risk_Level"] == "Medium":
        return np.random.normal(0.0007, 0.012)
    else:
        return np.random.normal(0.0004, 0.005)

merged["Daily_Return"] = merged.apply(simulate_returns, axis=1)

In [64]:
merged["Daily_Return"].describe()

count    1000.000000
mean        0.001199
std         0.015056
min        -0.039785
25%        -0.007430
50%         0.000941
75%         0.008386
max         0.078055
Name: Daily_Return, dtype: float64

In [65]:
merged["Daily_PnL"] = merged["Portfolio_Value_USD"] * merged["Daily_Return"]

In [66]:
merged["Daily_PnL"].describe()

count     1000.000000
mean       150.962115
std       5142.949463
min     -59281.263670
25%      -1008.772679
50%         24.615389
75%       1236.636235
max      28200.160995
Name: Daily_PnL, dtype: float64

In [67]:
VaR_95 = np.percentile(merged["Daily_PnL"], 5)
VaR_95

np.float64(-6915.8826899138985)

In [68]:
total_value = merged["Portfolio_Value_USD"].sum()
(VaR_95 / total_value) * 100

np.float64(-0.0027993274788188187)

In [69]:
total_portfolio_value = merged["Portfolio_Value_USD"].sum()
total_portfolio_value

np.float64(247055149.57585698)

In [94]:
#Monte Carlo Simulation

np.random.seed(42)
simulated_portfolio_pnl = []

for i in range(1000):
    simulated_returns = []
    for index, row in merged.iterrows():
        if row["Risk_Level"] == "High":
            r = np.random.normal(0.001, 0.02)
        elif row["Risk_Level"] == "Medium":
            r = np.random.normal(0.0007, 0.012)
        else:
            r = np.random.normal(0.0004, 0.005)

        simulated_returns.append(r)

simulated_returns = np.array(simulated_returns)
    
pnl = np.sum(merged["Portfolio_Value_USD"].values * simulated_returns)
    
simulated_portfolio_pnl.append(pnl)

simulated_portfolio_pnl = np.array(simulated_portfolio_pnl)

In [72]:
MC_VaR_95 = np.percentile(simulated_portfolio_pnl, 5)
MC_VaR_95

np.float64(-21997.397436119274)

In [73]:
(MC_VaR_95 / total_portfolio_value) * 100

np.float64(-0.008903840892968348)

In [93]:
#95% VaR threshold from Monte Carlo
var_threshold = MC_VaR_95

#losses worse than VaR
tail_losses = simulated_portfolio_pnl[
    simulated_portfolio_pnl <= var_threshold
]
#Expected Shortfall
CVaR_95 = tail_losses.mean()
CVaR_95

/var/folders/gh/h6dxdxzx5cv8zkl09fgm_2zm0000gn/T/ipykernel_2558/1299875423.py:9: RuntimeWarning: Mean of empty slice.
  CVaR_95 = tail_losses.mean()
/opt/anaconda3/lib/python3.13/site-packages/numpy/_core/_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


np.float64(nan)

In [75]:
(CVaR_95 / total_portfolio_value) * 100

np.float64(-0.008903840892968348)

In [76]:
len(tail_losses)

1

In [102]:
import numpy as np

#Risk mean and volatility mapping
risk_vol_map = {
    "High": (0.001, 0.02),
    "Medium": (0.0007, 0.012),
    "Low": (0.0004, 0.005)
}

#Mapping mean & std per portfolio row
means = merged["Risk_Level"].map(lambda x: risk_vol_map[x][0]).values
stds = merged["Risk_Level"].map(lambda x: risk_vol_map[x][1]).values

#Portfolio values
portfolio_values = merged["Portfolio_Value_USD"].values

In [103]:
np.random.seed(42)
simulations = 10000  

random_returns = np.random.normal(loc=means, scale=stds, size=(simulations, len(portfolio_values)))
# Compute portfolio PnL for each simulation
simulated_portfolio_pnl = np.sum(random_returns * portfolio_values, axis=1)

In [104]:
# 95% Value at Risk
VaR_95 = np.percentile(simulated_portfolio_pnl, 5)

# Conditional VaR (Expected Shortfall)
CVaR_95 = simulated_portfolio_pnl[simulated_portfolio_pnl <= VaR_95].mean()

print("VaR 95%:", VaR_95)
print("CVaR 95%:", CVaR_95)

VaR 95%: -85408.80610522475
CVaR 95%: -157429.01068213987


In [105]:
# Double volatilities
high_vol_map = {
    "High": (0.001, 0.04),
    "Medium": (0.0007, 0.024),
    "Low": (0.0004, 0.01)
}

means_stress = merged["Risk_Level"].map(lambda x: high_vol_map[x][0]).values
stds_stress = merged["Risk_Level"].map(lambda x: high_vol_map[x][1]).values

#Monte Carlo with doubled vol
random_returns_stress = np.random.normal(
    loc=means_stress,
    scale=stds_stress,
    size=(simulations, len(portfolio_values))
)

simulated_portfolio_pnl_stress = np.sum(random_returns_stress * portfolio_values, axis=1)

#VaR & CVaR for stressed market
VaR_stress = np.percentile(simulated_portfolio_pnl_stress, 5)
CVaR_stress = simulated_portfolio_pnl_stress[simulated_portfolio_pnl_stress <= VaR_stress].mean()

print("Stress VaR 95%:", VaR_stress)
print("Stress CVaR 95%:", CVaR_stress)

Stress VaR 95%: -354555.5263420011
Stress CVaR 95%: -501492.8093334981


In [106]:
VaR_threshold = np.percentile(simulated_portfolio_pnl, 5)

tail_indices = simulated_portfolio_pnl <= VaR_threshold

print("Number of tail scenarios:", tail_indices.sum())

Number of tail scenarios: 500


In [107]:
random_returns_matrix = np.random.normal(
    loc=means, 
    scale=stds, 
    size=(len(simulated_portfolio_pnl), len(portfolio_values))
)

portfolio_pnl_matrix = random_returns_matrix * portfolio_values

In [108]:
tail_pnl_matrix = portfolio_pnl_matrix[tail_indices]

In [116]:
import pandas as pd

segments = merged["Client_Segment"].values

tail_df = pd.DataFrame(tail_pnl_matrix, columns=segments)
segment_tail_loss = tail_df.T.groupby(level=0).sum().T

avg_segment_tail_loss = segment_tail_loss.mean()
print("Average Tail Loss per Segment (Normal Market):")
print(avg_segment_tail_loss)

Average Tail Loss per Segment (Normal Market):
HNI              73142.440198
Institutional    36179.042714
Retail           79203.133463
dtype: float64


In [117]:
VaR_stress_threshold = np.percentile(simulated_portfolio_pnl_stress, 5)
tail_indices_stress = simulated_portfolio_pnl_stress <= VaR_stress_threshold

print("Number of tail scenarios (Stress):", tail_indices_stress.sum())

Number of tail scenarios (Stress): 500


In [118]:
random_returns_matrix_stress = np.random.normal(
    loc=means_stress,
    scale=stds_stress,
    size=(len(simulated_portfolio_pnl_stress), len(portfolio_values))
)

portfolio_pnl_matrix_stress = random_returns_matrix_stress * portfolio_values

In [119]:
tail_pnl_matrix_stress = portfolio_pnl_matrix_stress[tail_indices_stress]

In [120]:
# Map segments as column names
segments = merged["Client_Segment"].values

# Convert to DataFrame
tail_df_stress = pd.DataFrame(tail_pnl_matrix_stress, columns=segments)

# Group by segment (modern way, no warnings)
segment_tail_loss_stress = tail_df_stress.T.groupby(level=0).sum().T

# Average tail loss per segment
avg_segment_tail_loss_stress = segment_tail_loss_stress.mean()
print("Average Tail Loss per Segment (Stress Market):")
print(avg_segment_tail_loss_stress)

Average Tail Loss per Segment (Stress Market):
HNI              75157.300586
Institutional    29625.306929
Retail           82726.981545
dtype: float64


In [121]:
import pandas as pd

# Normal Market Data
df_normal = pd.DataFrame({
    "Segment": avg_segment_tail_loss.index,
    "Avg_Tail_Loss": avg_segment_tail_loss.values,
    "Scenario": "Normal Market"
})

# Stress Market Data
df_stress = pd.DataFrame({
    "Segment": avg_segment_tail_loss_stress.index,
    "Avg_Tail_Loss": avg_segment_tail_loss_stress.values,
    "Scenario": "Stress Market"
})

#Combine both scenarios
df_tableau = pd.concat([df_normal, df_stress], ignore_index=True)

#sort by Scenario and Segment
df_tableau = df_tableau.sort_values(by=["Scenario", "Segment"]).reset_index(drop=True)

In [122]:
df_tableau.to_csv("Segment_Tail_Loss_Tableau.csv", index=False)
print("Tableau CSV saved successfully!")

Tableau CSV saved successfully!


In [131]:
risk_summary = pd.DataFrame({
    "Scenario": ["Normal Market", "Stress Market"],
    "VaR_95": [VaR_95, VaR_stress],
    "CVaR_95": [CVaR_95, CVaR_stress]
})

risk_summary

,Scenario,VaR_95,CVaR_95
0,Normal Market,-85408.806105,-157429.010682
1,Stress Market,-354555.526342,-501492.809333


In [132]:
risk_summary.to_csv("risk_summary.csv", index=False)